In [8]:
from giotto_tda.examples.data.generate_datasets import make_gravitational_waves
from pathlib import Path
import numpy as np

R = 0.65
n_signals = 1000
DATA = Path("./giotto_tda/examples/data")

noisy_signals, gw_signals, labels = make_gravitational_waves(
    path_to_data=DATA, n_signals=n_signals, r_min=R, r_max=R, n_snr_values=1
)

print(f"Number of noisy signals: {len(noisy_signals)}")
print(f"Number of timesteps per series: {len(noisy_signals[0])}")

Number of noisy signals: 1000
Number of timesteps per series: 8692


In [9]:
noisy_signals[0].shape

(8692,)

In [10]:
gw_signals[0].shape

(8192,)

In [11]:
padded_gw_signal = np.pad(gw_signals[0], (0,len(noisy_signals[0]) - len(gw_signals[0])), "constant",constant_values= 0).reshape(1, -1)

In [12]:
padded_gw_signal.shape

(1, 8692)

In [ ]:
labels

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# get the index corresponding to the first pure noise time series
background_idx = np.argmin(labels)
# get the index corresponding to the first noise + gravitational wave time series
signal_idx = np.argmax(labels)

ts_noise = noisy_signals[background_idx]
ts_background = noisy_signals[signal_idx]
ts_signal = gw_signals[signal_idx]

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=list(range(len(ts_noise))), y=ts_noise, mode="lines", name="noise"),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=list(range(len(ts_background))),
        y=ts_background,
        mode="lines",
        name="background",
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Scatter(x=list(range(len(ts_signal))), y=ts_signal, mode="lines", name="signal"),
    row=1,
    col=2,
)
fig.show()

In [ ]:
from gtda.time_series import SingleTakensEmbedding
embedding_dimension = 200
embedding_time_delay = 10
stride = 10
embedder = SingleTakensEmbedding(
    parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
)
#signaal++noise
y_gw_embedded = embedder.fit_transform(noisy_signals[signal_idx])
y_gw_embedded.shape

In [ ]:
embedding_dimension = 200
embedding_time_delay = 10
stride = 10

embedder = SingleTakensEmbedding(
    parameters_type="search", n_jobs=6, time_delay=embedding_time_delay, dimension=embedding_dimension, stride=stride
)

y_noise_embedded = embedder.fit_transform(noisy_signals[background_idx])
y_noise_embedded.shape

In [ ]:
import mtd
n = 100
barcs = [mtd.calc_cross_barcodes(y_gw_embedded, y_gw_embedded, batch_size1 = 300, batch_size2 = 600, pdist_device = "cuda", is_plot = False) for _ in range(n)]

In [ ]:
mtd_list_gw = [mtd.get_score(barc, 1, 'sum_length') for barc in barcs]

In [ ]:
import mtd
n = 100
barcs = [mtd.calc_cross_barcodes(y_noise_embedded, y_noise_embedded, batch_size1 = 100, batch_size2 = 500, pdist_device = "cuda", is_plot = False) for _ in range(n)]

In [ ]:
mtd_list_noise = [mtd.get_score(barc, 1, 'sum_length') for barc in barcs]

In [ ]:
import mtd
n = 100
barcs = [mtd.calc_cross_barcodes(y_gw_embedded, y_noise_embedded, batch_size1 = 100, batch_size2 = 500, pdist_device = "cuda", is_plot = False) for _ in range(n)]

In [ ]:
mtd_list_gw_noise = [mtd.get_score(barc, 1, 'sum_length') for barc in barcs]

In [ ]:
import mtd
n = 100
barcs = [mtd.calc_cross_barcodes(y_noise_embedded , y_gw_embedded, batch_size1 = 100, batch_size2 = 500, pdist_device = "cuda", is_plot = False) for _ in range(n)]

In [ ]:
mtd_list_noise_gw = [mtd.get_score(barc, 1, 'sum_length') for barc in barcs]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(mtd_list_noise, label="noise")
sns.kdeplot(mtd_list_gw,label="gw")
sns.kdeplot(mtd_list_gw_noise,label="gw_noise")
sns.kdeplot(mtd_list_noise_gw,label="noise_gw")
plt.legend()

---

Let's check how it works on many examples

In [ ]:
from gtda.time_series import TakensEmbedding
import numpy as np

embedding_dimension = 200
embedding_time_delay = 10
stride = 10

embedder = TakensEmbedding(time_delay=embedding_time_delay,
                           dimension=embedding_dimension,
                           stride=stride)

In [ ]:
features = np.array(embedder.fit_transform(noisy_signals))

In [ ]:
features.shape

In [ ]:
noised_gw = features[labels==1]

In [ ]:
y_noise_embedded = features[0]

In [ ]:
noise = features[labels==0]

In [ ]:
import mtd
from tqdm import tqdm

mtd_noised_gw = []

for cloud in tqdm(noised_gw):
    mtd_noised_gw.append(mtd.calc_cross_barcodes(y_noise_embedded , cloud, batch_size1 = 100, batch_size2 = 500, pdist_device = "cuda", is_plot = False))

In [ ]:
mtd_list_noise_gw = [mtd.get_score(barc, 1, 'sum_length') for barc in mtd_noised_gw]

In [ ]:
import mtd
from tqdm import tqdm

mtd_noised= []

for cloud in tqdm(noise):
    mtd_noised.append(mtd.calc_cross_barcodes(y_noise_embedded , cloud, batch_size1 = 100, batch_size2 = 500, pdist_device = "cuda", is_plot = False))

In [ ]:
mtd_list_noise_all = [mtd.get_score(barc, 1, 'sum_length') for barc in mtd_noised]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(mtd_list_noise, label="noise")
plt.scatter(mtd_list_noise_gw,np.zeros(len(mtd_list_noise_gw)),c = "red", label = "noise_gw")
plt.scatter(mtd_list_noise_all,np.zeros(len(mtd_list_noise_all)),c = "green", label = "noise_noise")
plt.legend()

---

Let's try add PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
features_pca = [pca.fit_transform(cloud) for cloud in features]

In [ ]:
features_pca = np.array(features_pca)

In [ ]:
noised_gw_pca = features_pca[labels==1]
noise_pca = features_pca[labels==0]
y_noise_embedded_pca = features_pca[0]

In [ ]:
import mtd
from tqdm import tqdm

mtd_noised_gw_pca = []

for cloud in tqdm(noised_gw_pca):
    mtd_noised_gw_pca.append(mtd.calc_cross_barcodes(y_noise_embedded_pca , cloud, batch_size1 = 100, batch_size2 = 500, pdist_device = "cuda", is_plot = False))

In [ ]:
mtd_list_noise_gw_pca = [mtd.get_score(barc, 1, 'sum_length') for barc in mtd_noised_gw_pca]

In [ ]:
import mtd
from tqdm import tqdm

mtd_noised_pca= []

for cloud in tqdm(noise_pca):
    mtd_noised_pca.append(mtd.calc_cross_barcodes(y_noise_embedded_pca , cloud, batch_size1 = 100, batch_size2 = 500, pdist_device = "cuda", is_plot = False))

In [ ]:
mtd_list_noise_all_pca = [mtd.get_score(barc, 1, 'sum_length') for barc in mtd_noised_pca]

In [ ]:
n = 100
barcs = [mtd.calc_cross_barcodes(y_noise_embedded_pca, y_noise_embedded_pca, batch_size1 = 100, batch_size2 = 500, pdist_device = "cuda", is_plot = False) for _ in range(n)]

In [ ]:
mtd_list_noise_pca = [mtd.get_score(barc, 1, 'sum_length') for barc in barcs]

In [ ]:
n = 100
barcs = [mtd.calc_cross_barcodes(features_pca[1], features_pca[1], batch_size1 = 100, batch_size2 = 500, pdist_device = "cuda", is_plot = False) for _ in range(n)]
mtd_list_gw_pca = [mtd.get_score(barc, 1, 'sum_length') for barc in barcs]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.kdeplot(mtd_list_noise_pca, label="noise")
sns.kdeplot(mtd_list_gw_pca, label="gw")
plt.scatter(mtd_list_noise_gw_pca,np.zeros(len(mtd_list_noise_gw_pca)),c = "red", label = "noise_gw")
plt.scatter(mtd_list_noise_all_pca,np.zeros(len(mtd_list_noise_all_pca)),c = "green", label = "noise_noise")
plt.legend()

Предположительно проблема в том, что облака шумовые генерируются из разных временных рядов, чья топология может отличаться. В задаче с цифрами, мы имел 5 облаков всего и могли отличить двуу разных подоблака с помощью этого меетода. Но тут у нас 100 разных облаков, которые сами по сеебее отличаются, и пока не удаается их категоризировать с помощью MTD.

---

### Now we want to reproduce author's experiments but with one substantial change. We will repllace barcode to cross-barcode

In [14]:
from scipy.stats import entropy

def my_entropy(cross_barcodes, normalize = False):
    pers_entropy = np.zeros((len(cross_barcodes),2))

    sum_lifespan = np.zeros(len(cross_barcodes))

    
    for barcode_idx in range(len(cross_barcodes)):
        for hom_dim in [0, 1]:
            lifespan_sums = cross_barcodes[barcode_idx][hom_dim][:, 1] - cross_barcodes[barcode_idx][hom_dim][:, 0]
            sum_lifespan[barcode_idx] += np.sum(lifespan_sums)
            entropy_dim = entropy(lifespan_sums, base = 2)
            
            pers_entropy[barcode_idx][hom_dim] = entropy_dim

    if normalize:
        pers_entropy /= np.log2(sum_lifespan[...,None])
        
    return pers_entropy
        
        
        

In [16]:
from gtda.diagrams import PersistenceEntropy, Scaler
from gtda.homology import VietorisRipsPersistence
from gtda.time_series import TakensEmbedding
from sklearn.decomposition import PCA
import mtd

embedding_dimension = 200
embedding_time_delay = 10
stride = 10

embedder = TakensEmbedding(time_delay=embedding_time_delay,
                           dimension=embedding_dimension,
                           stride=stride)

batch_pca = PCA(n_components=3)

persistence = mtd.calc_cross_barcodes 
## желательно дописать обертку преварщающую выход в тензор: ndarray of shape (n_samples, n_features, 3)
# Input data. Array of persistence diagrams, each a collection of
# triples [b, d, q] 

# scaling = Scaler()

# entropy = PersistenceEntropy(normalize=True, nan_fill_value=-10)


# steps = [("embedder", embedder),
#          ("pca", batch_pca),
#          ("persistence", persistence),
#          ("scaling", scaling),
#          ("entropy", entropy)]
# topological_transfomer = Pipeline(steps)

In [18]:
features = np.array(embedder.fit_transform(noisy_signals))

features = [batch_pca.fit_transform(cloud) for cloud in features]

In [20]:
padded_gw = batch_pca.fit_transform(np.array(embedder.transform(padded_gw_signal))[0])

In [21]:
first_noise = features[0]
first_gw = features[1]

In [52]:
import mtd
from tqdm import tqdm

cross_barcodes = []

for cloud in tqdm(features):
    cross_barcodes.append(persistence(first_noise , cloud, batch_size1 = 600, batch_size2 = 600, pdist_device = "cuda", is_plot = False))

entropyes_noise = my_entropy(cross_barcodes, normalize=False)
mtd_noise = [mtd.get_score(barc, 1, 'sum_length') for barc in cross_barcodes]

100%|███████████████████████████████████████| 1000/1000 [33:32<00:00,  2.01s/it]


In [53]:
import mtd
from tqdm import tqdm

cross_barcodes = []

for cloud in tqdm(features):
    cross_barcodes.append(persistence(cloud, first_noise, batch_size1 = 600, batch_size2 = 600, pdist_device = "cuda", is_plot = False))

entropyes_noise_rw = my_entropy(cross_barcodes, normalize=False)
mtd_noise_rw = [mtd.get_score(barc, 1, 'sum_length') for barc in cross_barcodes]

100%|███████████████████████████████████████| 1000/1000 [33:06<00:00,  1.99s/it]


In [54]:
import mtd
from tqdm import tqdm

cross_barcodes = []

for cloud in tqdm(features):
    cross_barcodes.append(persistence(first_gw , cloud, batch_size1 = 600, batch_size2 = 600, pdist_device = "cuda", is_plot = False))

entropyes_gw = my_entropy(cross_barcodes, normalize=False)
mtd_gw = [mtd.get_score(barc, 1, 'sum_length') for barc in cross_barcodes]

100%|███████████████████████████████████████| 1000/1000 [33:16<00:00,  2.00s/it]


In [55]:
import mtd
from tqdm import tqdm

cross_barcodes = []

for cloud in tqdm(features):
    cross_barcodes.append(persistence(cloud, first_gw, batch_size1 = 600, batch_size2 = 600, pdist_device = "cuda", is_plot = False))

entropyes_gw_rw = my_entropy(cross_barcodes, normalize=False)
mtd_gw_rw = [mtd.get_score(barc, 1, 'sum_length') for barc in cross_barcodes]

100%|███████████████████████████████████████| 1000/1000 [33:43<00:00,  2.02s/it]


In [56]:
import mtd
from tqdm import tqdm

cross_barcodes = []

for cloud in tqdm(features):
    cross_barcodes.append(persistence(padded_gw, cloud, batch_size1 = 600, batch_size2 = 600, pdist_device = "cuda", is_plot = False))

entropyes_gw_pure = my_entropy(cross_barcodes, normalize=False)
mtd_gw_pure = [mtd.get_score(barc, 1, 'sum_length') for barc in cross_barcodes]

100%|███████████████████████████████████████| 1000/1000 [38:03<00:00,  2.28s/it]


In [57]:
import mtd
from tqdm import tqdm

cross_barcodes = []

for cloud in tqdm(features):
    cross_barcodes.append(persistence(cloud, padded_gw, batch_size1 = 600, batch_size2 = 500, pdist_device = "cuda", is_plot = False))

entropyes_gw_pure_rw = my_entropy(cross_barcodes, normalize=False)
mtd_gw_pure_rw = [mtd.get_score(barc, 1, 'sum_length') for barc in cross_barcodes]

100%|███████████████████████████████████████| 1000/1000 [30:41<00:00,  1.84s/it]


In [59]:
mtd_noise = np.array(mtd_noise).reshape(-1,1)
mtd_noise_rw = np.array(mtd_noise_rw).reshape(-1,1)
mtd_gw = np.array(mtd_gw).reshape(-1,1)
mtd_gw_rw = np.array(mtd_gw_rw).reshape(-1,1)
mtd_gw_pure = np.array(mtd_gw_pure).reshape(-1,1)
mtd_gw_pure_rw = np.array(mtd_gw_pure_rw).reshape(-1,1)

---

In [270]:
# tmp = cross_barcodes[1][0]
# tmp = np.concatenate((tmp, np.repeat(0,len(tmp))[...,None]),axis = 1)

# tmp = tmp[None,...]

# for item in cross_barcodes:
#     for dim in item:
#         tmp = np.concatenate((tmp, np.concatenate((dim, np.repeat(0,len(dim))[...,None]),axis = 1)[None,...] ),axis = 0)

In [271]:
# scaled_cross_barcodes = scaling.fit_transform(cross_barcodes)

In [272]:
# entropy_of_cross_barcodes = [entropy.fit_transform(item) for item in cross_barcodes]

### Let's try to compute more features, as we have such opportunity

In [96]:
mtds = np.concatenate([mtd_noise, mtd_gw,
                            mtd_noise_rw, mtd_gw_rw,
                            mtd_gw_pure, mtd_gw_pure_rw
                           ], axis = 1)*10**18

In [97]:
entropyes = np.concatenate([entropyes_noise, entropyes_gw,
                            entropyes_noise_rw, entropyes_gw_rw,
                            entropyes_gw_pure, entropyes_gw_pure_rw
                           ], axis = 1)

In [98]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    entropyes, labels, test_size=0.2, random_state=42
)

In [99]:
from sklearn.metrics import accuracy_score, roc_auc_score


def print_scores(fitted_model):
    res = {
        "Accuracy on train:": accuracy_score(fitted_model.predict(X_train), y_train),
        "ROC AUC on train:": roc_auc_score(
            y_train, fitted_model.predict_proba(X_train)[:, 1]
        ),
        "Accuracy on valid:": accuracy_score(fitted_model.predict(X_valid), y_valid),
        "ROC AUC on valid:": roc_auc_score(
            y_valid, fitted_model.predict_proba(X_valid)[:, 1]
        ),
    }

    for k, v in res.items():
        print(k, round(v, 3))

In [100]:
from sklearn.linear_model import LogisticRegression

X_train, X_valid, y_train, y_valid = train_test_split(
    entropyes, labels, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)
print_scores(model)

Accuracy on train: 0.685
ROC AUC on train: 0.747
Accuracy on valid: 0.72
ROC AUC on valid: 0.766


In [101]:
from sklearn.linear_model import LogisticRegression

X_train, X_valid, y_train, y_valid = train_test_split(
    mtds, labels, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)
print_scores(model)

Accuracy on train: 0.678
ROC AUC on train: 0.731
Accuracy on valid: 0.72
ROC AUC on valid: 0.793


In [102]:
from sklearn.linear_model import LogisticRegression

X_train, X_valid, y_train, y_valid = train_test_split(
    np.concatenate([entropyes, mtds],axis = 1), labels, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)
print_scores(model)

Accuracy on train: 0.685
ROC AUC on train: 0.751
Accuracy on valid: 0.735
ROC AUC on valid: 0.802


---